# Food Classifier Project
#### Created by Thierry LAPLANCHE

Contents :
1. Import libraries
2. Install Azure Machine Learning SDK for Python
3. Create a Machine Learning workspace
4. Connect to your ML workspace
5. Register the model from TFSaved Model folder
6. Create an inference configuration
7. Deploy the model to Azure Container Instances
8. Test the endpoint
9. Update the model

## 1. Import libraries

In [1]:
# TensorFlow
import tensorflow as tf

# Azure ML libraries
from azureml.core import Workspace
import urllib.request
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice, Webservice

Enter your Azure subscription ID in the following cell.

In [2]:
subscription_id = 'XXXXXXXXXXXXXXXXXXXXXX' # enter your own Azure subscription ID (https://portal.azure.com/#blade/Microsoft_Azure_Billing/SubscriptionsBlade)
location='southeastasia' # location where to host the workspace and the model
workspace_name='foodidentifierworkspace' # name of the workspace to create (will result in an error if name already exists)
resourcegroup_name='foodidentifierresourcegroup' # name of the resource group (will be created automatically if non-existent)
model_name='foodidentifiermodel' # name of the model to deploy
service_name='foodidentifierservice' # name of the service once the model is deployed (can be the same name as the model)

## 2. Install Azure Machine Learning SDK for Python

Uncomment the next cell if you need to install Azure SDK.

In [3]:
#!pip install azureml-core

## 3. Create a Machine Learning workspace

Run this code only the first time, if you don't already have a Machine Learning workspace in your Azure environment.  
Otherwise, go to step 3.

In [ ]:
ws = Workspace.create(name=workspace_name,
               subscription_id=subscription_id,
               resource_group=resourcegroup_name,
               create_resource_group=True,
               location=location
               )

## 4. Connect to your ML workspace

Now that you have created an ML workspace, connect to it.

In [ ]:
ws = Workspace(subscription_id=subscription_id,
               resource_group=resourcegroup_name,
               workspace_name=workspace_name)

print("Workspace name: {}".format(ws.name) +
      "\nResource group: {}".format(ws.resource_group) +
      "\nLocation: {}".format(ws.location) +
      "\nSubscription ID: {}".format(ws.subscription_id))

## 5. Register the model from TFSavedModel folder

We will register the model saved locally to your Azure ML workspace.  
The model you have trained must be saved in the path defined in the following cell (by default : 'TFSavedModel')  
This step may take a few minutes.

In [33]:
# Register the model which is saved in the TFSavedModel folder
model = Model.register(ws,
                       model_name=model_name, # name of the model
                       model_path='./TFSavedModel', # local path to the model files
                       model_framework=Model.Framework.TENSORFLOW, # framework on which the model is based
                       model_framework_version=tf.__version__, # version of the TensorFlow framework
                       description='Model to identify food.') # description of the model

print('Name:', model.name)
print('Version:', model.version)

Registering model foodidentifiermodelbaseline
Name: foodidentifiermodelbaseline
Version: 1


## 6. Create an inference configuration

We need to configure your inference environment so that the model can be called as a Web Service from an external application.  
First, we will create a YAML configuration file which contains all the Python dependencies that are required to execute the model.  
Those are the libraries which are used in the entry script of your model.

In [34]:
# Create the dependencies configuration file myenv.yml
cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_conda_package('pillow')
cd.add_pip_package('tensorflow==2.3.0')
cd.add_pip_package('pickle-mixin')
cd.add_pip_package("azureml-contrib-services")
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

'myenv.yml'

Now we define the inference configuration and specifiy the location of the entry script.  
You might need to modify the number of CPU cores or memory allocated to the execution of your model, according to its complexity and the speed of execution you want.

In [35]:
# Create a deployment configuration
inference_env = Environment.from_conda_specification(name="inference_env", file_path="myenv.yml") # inference environment
inference_config = InferenceConfig(entry_script="entry_script.py", source_directory="./deployment", environment=inference_env) # inference configuration which specifies the entry script
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, # number of CPU cores
                                               memory_gb=1, # amount of memory
                                               description='Model to identify food.')

## 7. Deploy the model to Azure Container Instances

The model will de deployed to an isolated container instance with a unique endpoint (URI) to which you will call your model.  
This step may take over 10 minutes to complete.

In [36]:
# Deploy the model to Azure Container Instances
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig,
                           overwrite=True, # allows to overwrite an existing deployment
                           show_output=True)

service.wait_for_deployment(True)
print(service.state)
print(service.get_logs())

Models: ['foodidentifiermodelbaseline:1']
Entry script: deployment\entry_script.py
Environment dependencies: ['python=3.6.2', ordereddict([('pip', ['tensorflow==2.3.0', 'pickle-mixin', 'azureml-contrib-services', 'azureml-defaults'])]), 'numpy', 'pillow']
Environment docker image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1
CPU requirement: 1, Memory requirement: 1GB
Uploading dependency C:\Users\thier\AppData\Local\Temp\tmpds7_htwt\10318f7f.tar.gz.
Request submitted, please run wait_for_deployment(show_output=True) to get deployment status.
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-13 18:29:22+09:00 Creating Container Registry if not exists.
2021-07-13 18:29:22+09:00 Registering the environment.
2021-07-13 18:29:23+09:00 Use the existing image.
2021-07-13 18:29:24+09:00 Generating deployment configuration.
2021-07-

## 8. Test the endpoint

In [10]:
uri = service.scoring_uri

print("The URI of the endpoint is: {}".format(service.scoring_uri))

The URI of the endpoint is: http://be42e604-f4f7-46f8-819f-bb046c43338d.southeastasia.azurecontainer.io/score


We will use a tool called HTTPie to call our web service with a picture as a parameter.  
Uncomment the following line to install HTTPie tool.

In [11]:
#!python -m pip install --upgrade httpie

The command to call the web service from Jupyter Notebook is:  
!http --ignore-stdin -f POST http://uri_of_the_endpoint file@tile_of_the_picture.jpg  
You can replace the name of the picture 'test1.png' with a picture of your choice (saved in the same folder as this file) in the following line.

In [12]:
!http --ignore-stdin -f POST $uri file@test1.png > result.json

The web service returns JSON data. We saved it into 'result.json'.  
Let's see the results.

In [13]:
import json

# Open the results from the JSON file
with open('result.json') as json_file:
    prediction = json.load(json_file)

prediction

{'Bak kuh teh pork rib soup': 0.012148265726864338,
 'Baklava': 0.004917326848953962,
 'Char kway teow': 2.9208351861598203e-06,
 'Chilli crab': 2.0984433035664551e-07,
 'Hummus': 0.9747801423072815,
 'Kanafeh': 8.537168469047174e-05,
 'Rojak': 6.1727905631414615e-06,
 'Seafood hor fun': 1.6346919437637553e-05,
 'Shawarma': 0.008019939064979553,
 'Tabouleh': 2.322893669770565e-05}

The result is a dictionnary in which the keys are the classes (name of the dish) and the values are the probabilities.  
Let's sort the results by reverse order so that the item with highest probability comes first in the list.

In [96]:
sorted_prediction = [(k, prediction[k]) for k in sorted(prediction, key=prediction.get, reverse=True)]

sorted_prediction

[('Hummus', 0.9747801423072815),
 ('Bak kuh teh pork rib soup', 0.012148265726864338),
 ('Shawarma', 0.008019939064979553),
 ('Baklava', 0.004917326848953962),
 ('Kanafeh', 8.537168469047174e-05),
 ('Tabouleh', 2.322893669770565e-05),
 ('Seafood hor fun', 1.6346919437637553e-05),
 ('Rojak', 6.1727905631414615e-06),
 ('Char kway teow', 2.9208351861598203e-06),
 ('Chilli crab', 2.0984433035664551e-07)]

In [97]:
print('The top prediction for this picture is {} with a probability of {}'.format(sorted_prediction[0][0],sorted_prediction[0][1]))

The top prediction for this picture is Hummus with a probability of 0.9747801423072815


## 9. Update the model

If you retrain your model, you need to register the new model to your Azure ML workspace and link it to the existing endpoint.  
Again, your new model must be saved in the path specified in the following cell ('TFSavedModel' by default).  
The version of the model will be increased by Azure.  
This step may take a few minutes.

In [41]:
# Register the updated model which is saved in the TFSavedModel folder
new_model = Model.register(ws,
                       model_name="foodidentifiermodelbaseline", # name of the model
                       model_path='./TFSavedModel', # local path to the model files
                       model_framework=Model.Framework.TENSORFLOW, # framework on which the model is based
                       model_framework_version=tf.__version__, # version of the TensorFlow framework
                       description='Model to identify food.') # description of the model

print('Name:', new_model.name)
print('Version:', new_model.version)

Registering model foodidentifiermodelbaseline
Name: foodidentifiermodelbaseline
Version: 3


In [ ]:
# Retrieve the existing workspace
ws = Workspace(subscription_id=subscription_id,
               resource_group=resourcegroup_name,
               workspace_name=workspace_name)

print("Workspace name: {}".format(ws.name) +
      "\nResource group: {}".format(ws.resource_group) +
      "\nLocation: {}".format(ws.location) +
      "\nSubscription ID: {}".format(ws.subscription_id))

# Create a deployment configuration
inference_env = Environment.from_conda_specification(name="inference_env", file_path="myenv.yml") # inference environment
inference_config = InferenceConfig(entry_script="entry_script.py", source_directory="./deployment", environment=inference_env) # inference configuration which specifies the entry script

# Retrieve the existing service (endpoint)
service = Webservice(name="foodidentifierservicebaseline", workspace=ws)

In [43]:
# Update the existing endpoint with the new model (this step may take a few minutes)
service.update(models=[new_model], inference_config=inference_config)
service.wait_for_deployment(show_output=True)

print(service.state)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-13 21:44:53+09:00 Creating Container Registry if not exists.
2021-07-13 21:44:53+09:00 Registering the environment.
2021-07-13 21:44:54+09:00 Use the existing image.
2021-07-13 21:44:55+09:00 Generating deployment configuration.
2021-07-13 21:44:56+09:00 Submitting deployment to compute.
2021-07-13 21:44:58+09:00 Checking the status of deployment foodidentifierservicebaseline..
2021-07-13 21:55:07+09:00 Checking the status of inference endpoint foodidentifierservicebaseline.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
2021-07-13T12:54:32,668242600+00:00 - rsyslog/run 
2021-07-13T12:54:32,672586700+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-13T12:54:32,683411100+00:00 - iot-server/run 
2021-07-13T12:54:32,693424700+